In [133]:
import pandas as pd
import numpy as np
from scipy.special import expit

def logistic(x):
    return 1.0/(1 + expit(-x))

def logistic_deriv(x):
    return logistic(x) * (1 - logistic(x))

In [140]:
LR = 1   

I_dim = 1
H_dim = 5

epoch_count = 1

np.random.seed(1)
weights_ItoH = np.random.uniform(-1, 1, (I_dim, H_dim))
weights_HtoO = np.random.uniform(-1, 1, H_dim)

preActivation_H = np.zeros(H_dim)
postActivation_H = np.zeros(H_dim)
postActivation_O = 0

In [141]:
# Loading in trining data
training_data = pd.read_csv("songinput.csv")
target_output = training_data.no
training_data = training_data.drop(['no'], axis=1)

# Delete non-number columns
training_data = training_data.drop(['artist_name'], axis=1)
training_data = training_data.drop(['track_name'], axis=1)
training_data = training_data.drop(['track_id'], axis=1)
training_data = training_data.drop(['uri'], axis=1)
training_data = training_data.drop(['track_href'], axis=1)
training_data = training_data.drop(['analysis_url'], axis=1)
training_data = training_data.drop(['type'], axis=1)

training_data = training_data.drop(['popularity'], axis=1)
training_data = training_data.drop(['danceability'], axis=1)
training_data = training_data.drop(['energy'], axis=1)
training_data = training_data.drop(['key'], axis=1)
training_data = training_data.drop(['loudness'], axis=1)
training_data = training_data.drop(['mode'], axis=1)
training_data = training_data.drop(['speechiness'], axis=1)
training_data = training_data.drop(['acousticness'], axis=1)
training_data = training_data.drop(['instrumentalness'], axis=1)
training_data = training_data.drop(['liveness'], axis=1)
training_data = training_data.drop(['valence'], axis=1)
training_data = training_data.drop(['duration_ms'], axis=1)
training_data = training_data.drop(['time_signature'], axis=1)

training_data = np.asarray(training_data,dtype=np.float64)
training_count = len(training_data[:,0])

print(target_output)

0      1
1      4
2      8
3      3
4      9
5     21
6     14
7     27
8     28
9     29
10    31
11     2
Name: no, dtype: int64


In [142]:
# Loading in validation data
validation_data = pd.read_csv("songinput2.csv")
validation_output = validation_data.no
validation_data = validation_data.drop(['no'], axis=1)

# Delete non-number columns
validation_data = validation_data.drop(['artist_name'], axis=1)
validation_data = validation_data.drop(['track_name'], axis=1)
validation_data = validation_data.drop(['track_id'], axis=1)
validation_data = validation_data.drop(['uri'], axis=1)
validation_data = validation_data.drop(['track_href'], axis=1)
validation_data = validation_data.drop(['analysis_url'], axis=1)
validation_data = validation_data.drop(['type'], axis=1)

validation_data = validation_data.drop(['popularity'], axis=1)
validation_data = validation_data.drop(['danceability'], axis=1)
validation_data = validation_data.drop(['energy'], axis=1)
validation_data = validation_data.drop(['key'], axis=1)
validation_data = validation_data.drop(['loudness'], axis=1)
validation_data = validation_data.drop(['mode'], axis=1)
validation_data = validation_data.drop(['speechiness'], axis=1)
validation_data = validation_data.drop(['acousticness'], axis=1)
validation_data = validation_data.drop(['instrumentalness'], axis=1)
validation_data = validation_data.drop(['liveness'], axis=1)
validation_data = validation_data.drop(['valence'], axis=1)
validation_data = validation_data.drop(['duration_ms'], axis=1)
validation_data = validation_data.drop(['time_signature'], axis=1)
validation_data = np.asarray(validation_data,dtype=np.float64)
validation_count = len(validation_data[:,0])
print(validation_output)

0    39
1    45
Name: no, dtype: int64


In [143]:
#Training process

for epoch in range(epoch_count):
    for sample in range(training_count):
        for node in range(H_dim):
            preActivation_H[node] = np.dot(training_data[sample,:], weights_ItoH[:, node])
            postActivation_H[node] = logistic(preActivation_H[node])
            
        FE = postActivation_O - target_output[sample]
        
        for H_node in range(H_dim):
            S_error = FE * logistic_deriv(preActivation_O)
            gradient_HtoO = S_error * postActivation_H[H_node]
                       
            for I_node in range(I_dim):
                input_value = training_data[sample, I_node]
                gradient_ItoH = S_error * weights_HtoO[H_node] * logistic_deriv(preActivation_H[H_node]) * input_value
                
                weights_ItoH[I_node, H_node] -= LR * gradient_ItoH
                
            weights_HtoO[H_node] -= LR * gradient_HtoO
        preActivation_O = np.dot(postActivation_H, weights_HtoO)
        postActivation_O = logistic(preActivation_O)

In [146]:
#validation

correct_classification_count = 0
for sample in range(validation_count):
    for node in range(H_dim):
        preActivation_H[node] = np.dot(validation_data[sample,:], weights_ItoH[:, node])
        postActivation_H[node] = logistic(preActivation_H[node])
            
    preActivation_O = np.dot(postActivation_H, weights_HtoO)
    postActivation_O = logistic(preActivation_O)

    print("original value: {}".format(validation_output[sample]))
    print("predicted score: {}\n".format(postActivation_O))
#    if postActivation_O > 0.5:
#        output = 1
#    else:
#        output = 0     
#        
#    if output == validation_output[sample]:
#        correct_classification_count += 1

#print('Percentage of correct classifications:')
#print(correct_classification_count*100/validation_count)

original value: 39
predicted score: 0.9991843645134685

original value: 45
predicted score: 0.9991843645134685

